In [1]:
import os
import yaml
import pandas as pd
import numpy as np
import multiprocessing as mp
from functools import partial
import h3
from tsnecuda import TSNE
import math
import json
import gc
from collections import OrderedDict
##For merging sets of dictionaries
from mergedeep import merge as dict_merge
from mergedeep import  Strategy
import sys
import copy
import pickle
import multiprocessing as mp
from datetime import date
from dateutil.relativedelta import relativedelta

from tqdm import tqdm
import itertools
from collections import defaultdict
import random

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from datetime import date
import re

import geohash as gh

# import sys
# sys.path.insert(1, '../hve_challenger_train_sales_pipeline/hve_challenger_train_sale_dp/local_dependencies/')
# from geoTree.utils import percentiles_list
# from geoTree.geoTree import GeoTree

##Modeling Related Packages
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable

import gpytorch
import pytorch_forecasting
from pytorch_forecasting.models.temporal_fusion_transformer.sub_modules import GatedResidualNetwork, VariableSelectionNetwork, GatedLinearUnit, AddNorm ,GateAddNorm

from sklearn.preprocessing import RobustScaler, StandardScaler , OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight ,compute_sample_weight
from sklearn.inspection import PartialDependenceDisplay
from sklearn.base import BaseEstimator, TransformerMixin , _OneToOneFeatureMixin 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion 
from sklearn.utils.validation import check_is_fitted
#from sklearn.manifold import TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE
##Interpretability
import shap as shp
#from tsnecuda import TSNE

##Plotting Libs
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
##Path to local dependencies
#from torch_local.custom_layers import GMLPBlock, SegmentedMLP, self_attention
#from torch_local.loss_functions import loss_pc10_adaptive

##Model Tunning
import ray
from ray import tune,air
from ray.tune import JupyterNotebookReporter
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.bohb import TuneBOHB
from ray.tune.schedulers import ASHAScheduler
import pytorch_lightning as pl
from pytorch_lightning import trainer
from pytorch_lightning.loggers import TensorBoardLogger
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback
from ray.air import session
from pytorch_lightning.callbacks import ModelCheckpoint
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.lightning import (
    LightningTrainer,
    LightningConfigBuilder,
    LightningCheckpoint,
)
import pickle
import dill
from scipy.ndimage import gaussian_filter1d
from scipy.signal.windows import triang
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, pipeline,AutoModelForCausalLM


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("-----device:{}".format(device))
print("-----Pytorch version:{}".format(torch.__version__))
print(f"gpu available: {torch.cuda.is_available()}")

-----device:cpu
-----Pytorch version:1.12.1+cu102
gpu available: False


In [4]:
percentiles_list = [.01,.05,.1,.25,.5,.75,.9,.95,.99]

In [5]:
##Set some global Vars and functions

def pc10_calc(y, y_pred):    
    return round(np.mean((np.abs(y-y_pred)/y)<=0.1),3)


def pc10_accm(y, y_pred):    
    return np.sum((np.abs(y-y_pred)/y)<=0.1)


def pc10plus_accm_torch(y,y_pred):
    ratio = y_pred/y
    return torch.sum(ratio > 1.1)


def pc10_accm_torch(y,y_pred):
    diff = torch.abs(y-y_pred)/y
    return torch.sum(diff<= 0.1)

def median_bias(y, y_pred):
    return round(((y_pred - y) / y).median(), 3)

def pc10plus (y, y_pred):
    return round(np.mean((y_pred/y) > 1.1), 3)

def mape(y,y_pred):
    diff = np.abs((y-y_pred)/y)
    return round(np.mean(diff),3)

# for dataframe
def df_metric_wArg(df,colTrue,colPred):
    return pd.Series({'count'      : df.shape[0],
                      'pc10'      : pc10_calc(df.loc[:, colTrue], df.loc[:, colPred]),
                      'pc10+'     : pc10plus(df.loc[:, colTrue], df.loc[:, colPred]),
                      'median_bias': median_bias(df.loc[:, colTrue], df.loc[:, colPred]),
                      'mape' : mape(df.loc[:, colTrue], df.loc[:, colPred])
                     })

def adj_col_list(list_to_adj,ref_list,retain=True):
    ##If True keep elements in ref_list
    if retain:
        result = [col for col in list_to_adj if col in ref_list]
    else:
        result = [col for col in list_to_adj if col not in ref_list]
    return result

In [6]:
#Set the State
state = ['WA']
trgt_cnty = ['WA063']
time = '2023m05'
ple_size = 45

In [7]:
datapath = f'/home/jovyan/challenger/data'
artpath = f'/home/jovyan/challenger/artifacts'
print(datapath)
print(artpath)

/home/jovyan/challenger/data
/home/jovyan/challenger/artifacts


In [8]:
%%time
bluesky_data_train = pd.read_pickle(f'{datapath}/raw/bluesky_data_train_state_full_{time}_extended.pkl')
bluesky_data_train=bluesky_data_train[bluesky_data_train['final_state'].isin(state)].reset_index(drop=True)

CPU times: user 1min 7s, sys: 34.1 s, total: 1min 41s
Wall time: 1min 41s


In [9]:
bluesky_data_test =  pd.read_pickle(f'{datapath}/raw/bluesky_data_test_state_full_{time}_extended.pkl')
bluesky_data_test=bluesky_data_test[bluesky_data_test['final_state'].isin(state)].reset_index(drop=True)

In [10]:
bluesky_data_train['final_ntdtefnd'] = pd.to_datetime(bluesky_data_train['final_ntdtefnd'])
bluesky_data_train['ntdtefnd_lag1'] = pd.to_datetime(bluesky_data_train['ntdtefnd_lag1'])

bluesky_data_train['min_age'] = bluesky_data_train['final_ntdtefnd'].min()
bluesky_data_train['sale_age_toanchr'] = ((bluesky_data_train.final_ntdtefnd.dt.year -bluesky_data_train.min_age.dt.year ) * 12 +  bluesky_data_train.final_ntdtefnd.dt.month - bluesky_data_train.min_age.dt.month).astype(int)   
bluesky_data_train["sale_age_lag1"] = ((bluesky_data_train.ntdtefnd_lag1.dt.year-bluesky_data_train.min_age.dt.year ) * 12 + (bluesky_data_train.ntdtefnd_lag1.dt.month - bluesky_data_train.min_age.dt.month)).astype(int,errors='ignore')

bluesky_data_train['sale_qtr'] = bluesky_data_train.final_ntdtefnd.dt.quarter
bluesky_data_train['log_trgt'] = np.log(bluesky_data_train.final_purprice)

In [11]:
bluesky_data_train.groupby('sale_age_toanchr').size()

sale_age_toanchr
0      8742
1      6377
2      6729
3      9107
4     10712
5     11244
6     13204
7     12173
8     12509
9     11730
10    11332
11    10149
12     8933
13     5861
14     6715
15    10027
16    10105
17    10707
18    10839
19     8517
20     9428
21     8453
22     6950
23     5393
24     4573
25     3939
26     5144
27     6712
28     6610
29     5823
dtype: int64

In [12]:
bluesky_data_test['final_ntdtefnd'] = pd.to_datetime(bluesky_data_test['final_ntdtefnd'])
bluesky_data_test['ntdtefnd_lag1'] = pd.to_datetime(bluesky_data_test['ntdtefnd_lag1'])

bluesky_data_test['min_age'] = bluesky_data_train['final_ntdtefnd'].min()
bluesky_data_test["sale_age_toanchr"] = ((bluesky_data_test.final_ntdtefnd.dt.year-bluesky_data_test.min_age.dt.year ) * 12 + (bluesky_data_test.final_ntdtefnd.dt.month - bluesky_data_test.min_age.dt.month)).astype(int)
bluesky_data_test["sale_age_lag1"] = ((bluesky_data_test.ntdtefnd_lag1.dt.year-bluesky_data_test.min_age.dt.year ) * 12 + (bluesky_data_test.ntdtefnd_lag1.dt.month - bluesky_data_test.min_age.dt.month)).astype(int,errors='ignore')

bluesky_data_test["sale_qtr"] =   bluesky_data_test.final_ntdtefnd.dt.quarter
bluesky_data_test['log_trgt'] = np.log(bluesky_data_test.final_purprice)

In [13]:
bluesky_data_test.groupby('sale_age_toanchr').size()

sale_age_toanchr
30    8800
dtype: int64

In [14]:
bluesky_data_train['clmls_cnt_tot_btrm'] = pd.to_numeric(bluesky_data_train['clmls_cnt_tot_btrm'])
bluesky_data_train['idmtax_nbr_lot_size_sq_ft'] = pd.to_numeric(bluesky_data_train['idmtax_nbr_lot_size_sq_ft'])
bluesky_data_train['clmls_nbr_area_bsmt'] = pd.to_numeric(bluesky_data_train['clmls_nbr_area_bsmt'])

bluesky_data_test['clmls_cnt_tot_btrm'] = pd.to_numeric(bluesky_data_test['clmls_cnt_tot_btrm'])
bluesky_data_test['idmtax_nbr_lot_size_sq_ft'] = pd.to_numeric(bluesky_data_test['idmtax_nbr_lot_size_sq_ft'])
bluesky_data_test['clmls_nbr_area_bsmt'] = pd.to_numeric(bluesky_data_test['clmls_nbr_area_bsmt'])

In [15]:
bluesky_data_train['H8']=bluesky_data_train.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],8), axis=1)
bluesky_data_train['H7']=bluesky_data_train.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],7), axis=1)
bluesky_data_train['H6']=bluesky_data_train.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],6), axis=1)
bluesky_data_train['H5']=bluesky_data_train.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],5), axis=1)
bluesky_data_train['H4']=bluesky_data_train.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],4), axis=1)
bluesky_data_train['H3']=bluesky_data_train.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],3), axis=1)
bluesky_data_train['H2']=bluesky_data_train.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],2), axis=1)


bluesky_data_test['H8']=bluesky_data_test.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],8), axis=1)
bluesky_data_test['H7']=bluesky_data_test.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],7), axis=1)
bluesky_data_test['H6']=bluesky_data_test.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],6), axis=1)
bluesky_data_test['H5']=bluesky_data_test.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],5), axis=1)
bluesky_data_test['H4']=bluesky_data_test.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],4), axis=1)
bluesky_data_test['H3']=bluesky_data_test.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],3), axis=1)
bluesky_data_test['H2']=bluesky_data_test.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'],2), axis=1)

In [16]:
##Get different hash lengths
bluesky_data_train['geohash4'] = bluesky_data_train.geohash5.apply(lambda x : x[:-1])
bluesky_data_train['geohash3'] = bluesky_data_train.geohash5.apply(lambda x : x[:-2])
bluesky_data_train['geohash2'] = bluesky_data_train.geohash5.apply(lambda x : x[:-3])
bluesky_data_test['geohash4'] = bluesky_data_test.geohash5.apply(lambda x : x[:-1])
bluesky_data_test['geohash3'] = bluesky_data_test.geohash5.apply(lambda x : x[:-2])
bluesky_data_test['geohash2'] = bluesky_data_test.geohash5.apply(lambda x : x[:-3])


In [17]:
bluesky_data_train['quarter'] = bluesky_data_train.final_ntdtefnd.apply(lambda x: pd.Timestamp(x).quarter-1)
bluesky_data_test['quarter'] = bluesky_data_test.final_ntdtefnd.apply(lambda x: pd.Timestamp(x).quarter-1)

In [18]:
clmls_desc_pub_rmrk_embed_train=torch.load(f'/home/jovyan/clmls_desc_pub_rmrk_embed_train_{state[0]}_{time}_extended.pkl')

In [19]:
clmls_desc_pub_rmrk_embed_test=torch.load(f'/home/jovyan/clmls_desc_pub_rmrk_embed_test_{state[0]}_{time}_extended.pkl')

In [20]:
ccmls_text_pub_lstg_cmnt_embed_train=torch.load(f'/home/jovyan/ccmls_text_pub_lstg_cmnt_train_{state[0]}_{time}_extended.pkl')

In [21]:
ccmls_text_pub_lstg_cmnt_embed_test=torch.load(f'/home/jovyan/ccmls_text_pub_lstg_cmnt_test_{state[0]}_{time}_extended.pkl')

In [22]:
ccmls_desc_aplnc_embed_train=torch.load(f'/home/jovyan/ccmls_desc_aplnc_train_{state[0]}_{time}_extended.pkl')

In [23]:
ccmls_desc_aplnc_embed_test=torch.load(f'/home/jovyan/ccmls_desc_aplnc_test_{state[0]}_{time}_extended.pkl')

In [24]:
ccmls_desc_eqpmnt_othr_embed_train=torch.load(f'/home/jovyan/ccmls_desc_eqpmnt_othr_train_{state[0]}_{time}_extended.pkl')

In [25]:
ccmls_desc_eqpmnt_othr_embed_test=torch.load(f'/home/jovyan/ccmls_desc_eqpmnt_othr_test_{state[0]}_{time}_extended.pkl')

In [26]:
bluesky_data_train['clmls_desc_pub_rmrk']=clmls_desc_pub_rmrk_embed_train.numpy().tolist() #[:,:75]

In [27]:
bluesky_data_test['clmls_desc_pub_rmrk']=clmls_desc_pub_rmrk_embed_test.numpy().tolist()#[:,:75]

In [28]:
bluesky_data_train['ccmls_text_pub_lstg_cmnt']=ccmls_text_pub_lstg_cmnt_embed_train.numpy().tolist()#[:,:75]

In [29]:
bluesky_data_test['ccmls_text_pub_lstg_cmnt']=ccmls_text_pub_lstg_cmnt_embed_test.numpy().tolist()#[:,:75]

In [30]:
bluesky_data_train['ccmls_desc_aplnc']=ccmls_desc_aplnc_embed_train.numpy().tolist()#[:,:75]

In [31]:
bluesky_data_test['ccmls_desc_aplnc']=ccmls_desc_aplnc_embed_test.numpy().tolist()#.numpy().tolist()[:,:75]

In [32]:
bluesky_data_train['ccmls_desc_eqpmnt_othr']=ccmls_desc_eqpmnt_othr_embed_train.numpy().tolist()#[:,:75]

In [33]:
bluesky_data_test['ccmls_desc_eqpmnt_othr']=ccmls_desc_eqpmnt_othr_embed_test.numpy().tolist()#[:,:75]

In [34]:
del clmls_desc_pub_rmrk_embed_train
del clmls_desc_pub_rmrk_embed_test
del ccmls_text_pub_lstg_cmnt_embed_train
del ccmls_text_pub_lstg_cmnt_embed_test
del ccmls_desc_aplnc_embed_train
del ccmls_desc_aplnc_embed_test
del ccmls_desc_eqpmnt_othr_embed_train
del ccmls_desc_eqpmnt_othr_embed_test

In [35]:
x_cols_ple_full = [
'ccmls_cnt_bdrm',
'ccmls_cnt_btrm',
'ccmls_cnt_ppty_stry',
'ccmls_cnt_unit',
'ccmls_dt_blt_yr',
'ccmls_nbr_gla',
'clmls_cnt_bdrm',
'clmls_cnt_tot_btrm',
'clmls_cnt_bldng_stry_cl',
'clmls_cnt_unit',
'clmls_cnt_grge_car',
'clmls_dt_blt_yr',
'clmls_nbr_area_livng',
'clmls_cnt_prkng_spce_open',
'clmls_cnt_prkng_spce_tot',
'fre_cnt_unit',
'fatax_amt_tot_asesd_val',
'fatax_amt_asesd_imprmt_val',
'fatax_amt_asesd_lnd_val',
'fatax_cnt_bdrm',
'fatax_cnt_bldng_stry',
'fatax_cnt_prkng_spce',
'fatax_cnt_tot_btrm_hve',
'fatax_cnt_unit',
'fatax_dt_eff_blt_yr',
'fatax_nbr_livng_sq_ft',
'fatax_nbr_lot_size_sq_ft',
'fatax_amt_tot_mkt_val',
'fatax_dt_cnsttn_yr',
'fatax_dt_tax_yr',
#'fatax_amt_calc_tot_val',
#'fatax_amt_calc_lnd_val',
#'fatax_amt_calc_imprmt_val'
#'fatax_amt_mkt_imprmt_val'
'idmtax_amt_asesd_tot_val',
'idmtax_amt_asesd_imprmt_val',
'idmtax_amt_asesd_lnd_val',
'idmtax_cnt_bdrm',
'idmtax_cnt_bldng_stry_drvd',
'idmtax_cnt_grge_car',
'idmtax_cnt_tot_btrm_hve',
'idmtax_cnt_unit',
'idmtax_dt_cnsttn_yr',
'idmtax_nbr_lot_size_sq_ft',
'idmtax_nbr_unvrsl_bldng_sq_ft_drvd',
'idmtax_amt_tot_mkt_val',
'idmtax_dt_tax_yr',
'lat',
'long',
'ucdp_cnt_prkng_spce',
'ucdp_cnt_tot_btrm_hve',
'ucdp_cnt_bdrm',
'ucdp_cnt_unit_hve',
'ucdp_effectiveage',
'ucdp_nbr_bldng_sq_ft_hve',
'ucdp_num_grge_hve',
'ucdp_quality_of_construction',
'ucdp_site_sqft',
'ucdp_stories',
'ucdp_year_built',
'ucdp_salescompsubjactualage',
#'mortgage_rate'
'mortgage_rate_lag1',
'mortgage_rate_lag2',
'mortgage_rate_lag3',
'mortgage_rate_lag4',
'mortgage_rate_lag5',
'mortgage_rate_lag6',
'avg_price_3m_bdrm_geohash6_orig',
'avg_price_6m_bdrm_geohash6_orig',
'avg_price_12m_bdrm_geohash6_orig',
#'avg_price_12m',
'purprice_lag1',
#'csushpinsa',
'csushpinsa_lag1',
'csushpinsa_lag2',
'csushpinsa_lag3',
'csushpinsa_lag4',
'csushpinsa_lag5',
'csushpinsa_lag6',
'ccmls_cnt_full_btrm','clmls_cnt_full_btrm','fatax_cnt_full_btrm','ucdp_cnt_full_btrm_hve',
'ccmls_cnt_half_btrm','clmls_cnt_half_btrm','ucdp_cnt_hlf_btrm_hve',
'ucdp_improvementsbasementarea', 'fatax_nbr_bsmt_sq_ft', 'ccmls_nbr_area_bsmt', 'clmls_nbr_area_bsmt',
'fatax_nbr_grge_sq_ft',
'clmls_amt_list_prce',
'ccmls_list_prce'
]


x_cols_scale_full = []

x_cols_emdd_full = [ 'final_zip','final_state','final_statecounty', 
               'geohash6','geohash5','geohash4' , 'geohash3','geohash2' ,'H8','H7','H6','H5','H4','H3','H2','quarter',
               'ccmls_type_ppty_hve','clmls_ppty_type','fnm_type_ppty_hve','fre_type_ppty_hve','idmtax_type_ppty_hve','ucdp_type_ppty_hve',
               'clmls_type_cond','ccmls_type_cond','ucdp_condition_rating',
               'idmtax_type_bldng_cnsttn_qual_hve','fatax_type_bldng_cnsttn_qual_hve', 'ucdp_qualityconstr_hve',
               'fatax_cd_absentee_ownr', 'idmtax_cd_ownr_ocpd',
               'fatax_type_bldng_styl', 'idmtax_type_bldng_styl', 'ccmls_type_home_styl', 'clmls_type_bldng_styl_cl',
               'fatax_type_grge','idmtax_type_grge', 'ccmls_type_grge', 'clmls_type_grge',
               'fatax_type_bsmt','idmtax_type_bsmt' , 'ccmls_type_bsmt','clmls_type_bsmt',
               #binary
               'ccmls_poor_cond_sale', 'clmls_poor_cond_sale', 'ccmls_renovated_sale','clmls_renovated_sale', 'ucdp_renovated_sale',
               'ccmls_ind_new_cntrctn', 'clmls_ind_new_cntrctn', 'ucdp_ind_new_cntrctn', 'ccmls_sub_cond_sale','clmls_sub_cond_sale'
               ]

x_cols_binary_full = [
                 
                 ]

x_cols_passthrough =['final_address','ntdtefnd_lag1','final_ntdtefnd','clmls_dttm_list_orig_cl','clmls_desc_pub_rmrk','ccmls_text_pub_lstg_cmnt','ccmls_desc_aplnc','ccmls_desc_eqpmnt_othr'
                     ,'sale_age_lag1','sale_age_toanchr','ccmls_dttm_mls_evnt','sale_age_toanchr_raw','final_statecounty_raw','final_state_raw']

target_col = ['log_trgt']

x_cols_full = x_cols_passthrough + x_cols_ple_full + x_cols_binary_full + x_cols_scale_full + x_cols_emdd_full  + target_col

In [36]:
bluesky_data_train['sale_age_toanchr_raw'] = bluesky_data_train.sale_age_toanchr
bluesky_data_test['sale_age_toanchr_raw'] = bluesky_data_test.sale_age_toanchr

bluesky_data_train['final_statecounty_raw'] = bluesky_data_train.final_statecounty
bluesky_data_test['final_statecounty_raw'] = bluesky_data_test.final_statecounty

bluesky_data_train['final_state_raw'] = bluesky_data_train.final_statecounty
bluesky_data_test['final_state_raw'] = bluesky_data_test.final_statecounty

In [37]:
##Subset to just the modeling columns
bluesky_data_train  = bluesky_data_train[x_cols_full].dropna(axis=1,how='all')

In [38]:
bluesky_data_train.shape

(258737, 153)

In [39]:
##Adjust the column definitions for cols that were dropped because all missing

x_cols_ple  =  adj_col_list(x_cols_ple_full,bluesky_data_train.columns.values.tolist())
x_cols_binary  = adj_col_list(x_cols_binary_full,bluesky_data_train.columns.values.tolist())
x_cols_scale = adj_col_list(x_cols_scale_full,bluesky_data_train.columns.values.tolist())
x_cols_emdd  = adj_col_list(x_cols_emdd_full,bluesky_data_train.columns.values.tolist())

x_cols = x_cols_passthrough + x_cols_ple + x_cols_binary + x_cols_scale + x_cols_emdd  + target_col

In [40]:
bluesky_data_test  = bluesky_data_test[x_cols]
bluesky_data_test.shape

(8800, 153)

In [41]:
bluesky_data_train[x_cols].isna().sum()/bluesky_data_train.shape[0]

final_address                         0.000000
ntdtefnd_lag1                         0.320391
final_ntdtefnd                        0.000000
clmls_dttm_list_orig_cl               0.275600
clmls_desc_pub_rmrk                   0.000000
ccmls_text_pub_lstg_cmnt              0.000000
ccmls_desc_aplnc                      0.000000
ccmls_desc_eqpmnt_othr                0.000000
sale_age_lag1                         0.320391
sale_age_toanchr                      0.000000
ccmls_dttm_mls_evnt                   0.841314
sale_age_toanchr_raw                  0.000000
final_statecounty_raw                 0.000000
final_state_raw                       0.000000
ccmls_cnt_bdrm                        0.842180
ccmls_cnt_btrm                        0.964079
ccmls_cnt_ppty_stry                   0.935460
ccmls_cnt_unit                        0.999992
ccmls_dt_blt_yr                       0.842802
ccmls_nbr_gla                         0.847436
clmls_cnt_bdrm                        0.280493
clmls_cnt_tot

In [42]:
gp_data = bluesky_data_train.copy()
test_df = bluesky_data_test.copy()

In [43]:
del bluesky_data_train
del bluesky_data_test
gc.collect()

0

In [44]:
##Define sklearn custom transformers##
class ColumnSelector(BaseEstimator, TransformerMixin,_OneToOneFeatureMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]

class seqPadder(BaseEstimator, TransformerMixin,_OneToOneFeatureMixin):
    """Pad Sequences to lengths specified by max lengths vector"""
    def __init__(self,max_lengths,pad_int):
        self.max_lengths = max_lengths
        self.pad_int = pad_int

    def padlist(self,l,colname):
        num = self.max_lengths[colname] - len(l)
        return l + num * [self.pad_int]

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        for col in X.columns.values.tolist():
            X[col] = X[col].apply(lambda x: self.padlist(x,col))
        return X

def convert_to_df(encode_data_list):
    dic={}
    for i in range(len(encode_data_list)):
        col=[encode_data_list[i][j] for j in range(len(encode_data_list[i]))]
        dic[i]=col

    return dic    

def convert_to_array(df):
    return np.array(
        [
            df.values[i][j].tolist() for i in range(df.shape[0]) for j in range(df.shape[1])
        ]
    ).reshape(df.shape[0], df.shape[1], -1)

class PLE(BaseEstimator, TransformerMixin, _OneToOneFeatureMixin):
    EPSILON = 1e-8
    JITTER = 0.05

    def __init__(self, num_bins=4):
        self.num_bins = num_bins    # Store the number of bins
        self.bin_boundaries = {}  # Initialize the bin boundaries

    def fit(self, X, y=None):
        # Fit the transformer to the data

        # Initialize an empty list to store the bin boundaries for each column
        quantiles = np.linspace(0, 1, self.num_bins + 1)
        self.feature_names = X.columns

        # Loop through each column and compute the bin boundaries
        for feature_name in self.feature_names:  # Iterate over columns
            column_data = X[feature_name]  # Extract the current column
            boundaries = column_data.quantile(quantiles) # Compute the bin boundaries for the current column
            self.bin_boundaries[feature_name] = boundaries  # Add the bin boundaries to the list

        return self

    def transform(self, X):
        # Transform the data using the fitted transformer
        # Loop through each column and perform piecewise linear encoding
        encode_data_list = []
        for feature_name in self.feature_names:  # Iterate over column
            column_data = X[feature_name].values  # Extract the current column
            column_bin_boundaries = np.array(self.bin_boundaries[feature_name])  # Get the bin boundaries for the current column

            # Initialize a matrix of all ones to store the encoded data
            encoded_data = np.ones([column_data.shape[0], self.num_bins])

            # Use np.digitize to find the bin indices for each data point
            bin_indices = np.digitize(column_data, column_bin_boundaries) - 1

            # compute numerator, adjust for edge case at max value
            # find the bin min for each data point
            bin_min = column_bin_boundaries[bin_indices]

            # for maximum data point, set bin min to second to last bin boundary
            bin_min[bin_indices == self.num_bins] = column_bin_boundaries[-2]

            # compute the numerator for each data point, x - bin[i-1]
            bin_numerator = column_data - bin_min

            # add jitter if column_data == bin_min, if column_data - bin_min == 0
            bin_numerator[bin_numerator == 0] = self.JITTER

            # Calculate the bin widths based on the bin boundaries
            bin_widths = np.diff(column_bin_boundaries)

            # adjust for edge case of last bin
            # for maximum data point, set bin width to last bin boundary
            idxs = bin_indices
            idxs[idxs == self.num_bins] = self.num_bins - 1

            # compute the demoninator for each data point: bin[i] - bin[i-1]
            bin_denominator = bin_widths[idxs]

            # Calculate the encoded value of each data point within the selected bin
            # Any np.inf values caused by divide by zero bin width, set to one.  This occurs in the
            # last bin when the last few bins have the same boundaries because of few unique values
            with np.errstate(divide="ignore"):
                encoded_values = bin_numerator / bin_denominator

            encoded_values[np.isinf(encoded_values)] = 1

            # Create a mask to store the encoded value in the corresponding column of encoded_data
            mask = np.zeros_like(encoded_data, dtype=bool)
            mask[np.arange(encoded_data.shape[0]), bin_indices] = True

            # Store the encoded value in the corresponding column of encoded_data
            encoded_data[mask] = encoded_values

            # Create mask to set all values after the column-specific bin index to 0
            mask = np.tile(np.arange(encoded_data.shape[1]), (encoded_data.shape[0], 1))
            mask = mask > bin_indices.reshape(-1, 1)
            encoded_data[mask] = 0

            # handle missing values
            encoded_data[np.isnan(column_data)] = 0

            # handle out of range data
            encoded_data[column_data < column_bin_boundaries[0]] = 0
            encoded_data[column_data > column_bin_boundaries[-1]] = 1
            encode_data_list.append(encoded_data.astype('float32'))

        dic = convert_to_df(encode_data_list)
        return pd.DataFrame(dic)
        # stack the encoded data encoded_data into a matrix that contains the piecewise linear encoding for each column
        # and reshape into form suitable to convert to a dataframe shape[# rows, # columns, # bins]
        #return np.array(encode_data_list,dtype='float32').transpose(1, 0, 2)
##Create a data preprocessor sklearn pipeline object ##
preprocessor = ColumnTransformer(transformers=[('ple', PLE(ple_size), x_cols_ple),
                                              ] +
                                              [('embed_'+col, 
                                                                         OrdinalEncoder(dtype=float,
                                                                                        categories= [sorted(gp_data[gp_data[col].isna()==False][col].unique().tolist())],
                                                                                        handle_unknown='use_encoded_value',
                                                                                        unknown_value=len(gp_data[gp_data[col].isna()==False][col].unique()),
                                                                                        encoded_missing_value=len(gp_data[gp_data[col].isna()==False][col].unique())
                                                                                       ),
                                                                         [col]
                                                                        ) for col in x_cols_emdd 
                                                                       ]+
                                              [
                                               ('slctr_passthrw', ColumnSelector(x_cols_passthrough), x_cols_passthrough),
                                               ('std_trgt', StandardScaler(), target_col)
                                              ],
                                n_jobs = 10 )

preprocessor.fit(gp_data.copy()) 

ColumnTransformer(n_jobs=10,
                  transformers=[('ple', PLE(num_bins=45),
                                 ['ccmls_cnt_bdrm', 'ccmls_cnt_btrm',
                                  'ccmls_cnt_ppty_stry', 'ccmls_cnt_unit',
                                  'ccmls_dt_blt_yr', 'ccmls_nbr_gla',
                                  'clmls_cnt_bdrm', 'clmls_cnt_tot_btrm',
                                  'clmls_cnt_bldng_stry_cl', 'clmls_cnt_unit',
                                  'clmls_cnt_grge_car', 'clmls_dt_blt_yr',
                                  'clmls_nbr_area_livng',
                                  'clmls_cnt_prkng_spce_open',
                                  'clml...
                                                             'KENT', ...]],
                                                dtype=<class 'float'>,
                                                encoded_missing_value=78,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=78),
                                 ['ucdp_qualityconstr_hve']),
                                ('embed_fatax_cd_absentee_ownr',
                                 OrdinalEncoder(categories=[['A', 'M', 'O', 'S',
                                                             'T']],
                                                dtype=<class 'float'>,
                                                encoded_missing_value=5,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=5),
                                 ['fatax_cd_absentee_ownr']), ...])

In [46]:
##Save the prprocessor
with open(f'{artpath}/{state[0]}_{time}_ple{ple_size}_processor_new_h3_txt_extended.dill', 'wb') as dill_file:
    dill.dump(preprocessor, dill_file)

In [47]:
##Clear memory from preprocessor build
gc.collect()

1263

In [48]:
##We will need mean and std of target scaler for calculating
##Pc10 later
trgt_mean = preprocessor.named_transformers_['std_trgt'].mean_[0]
trgt_s = preprocessor.named_transformers_['std_trgt'].scale_[0]
print(trgt_mean,trgt_s)

13.20246045067208 0.6203030147821884


In [51]:
%%time
##Preprocess the training data
X_train_s =  pd.DataFrame(preprocessor.transform(gp_data.copy()),index=gp_data.index,columns=[col.split('__')[1] for col in preprocessor.get_feature_names_out().tolist()])
print(X_train_s.shape)
##Save the preprocessed train file
X_train_s.to_pickle(f'{datapath}/preprocessed/{state[0]}_{time}_ple{ple_size}_X_train_s_new_h3_txt_extended.pkl')
##For now fill remaining NAN's which come from binaries with -1
#X_train_s.fillna(-1, inplace=True)
X_train_s.head()

(258737, 153)
CPU times: user 5min 34s, sys: 57.4 s, total: 6min 32s
Wall time: 8min 17s


,ccmls_cnt_bdrm,ccmls_cnt_btrm,ccmls_cnt_ppty_stry,ccmls_cnt_unit,ccmls_dt_blt_yr,ccmls_nbr_gla,clmls_cnt_bdrm,clmls_cnt_tot_btrm,clmls_cnt_bldng_stry_cl,clmls_cnt_unit,...,ccmls_text_pub_lstg_cmnt,ccmls_desc_aplnc,ccmls_desc_eqpmnt_othr,sale_age_lag1,sale_age_toanchr,ccmls_dttm_mls_evnt,sale_age_toanchr_raw,final_statecounty_raw,final_state_raw,log_trgt
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.05, 0.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.05, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [52]:
##Preprocess the Test data
X_test_s = pd.DataFrame(preprocessor.transform(test_df.copy()),index=test_df.index,columns=[col.split('__')[1] for col in preprocessor.get_feature_names_out().tolist()])
print(X_test_s.shape)
X_test_s.to_pickle(f'{datapath}/preprocessed/{state[0]}_{time}_ple{ple_size}_X_test_s_new_state_h3_txt_extended.pkl')
##For now fill remaining NAN's which come from binaries with -1
#X_test_s.fillna(-1, inplace=True)
X_test_s.head()

(8800, 153)


,ccmls_cnt_bdrm,ccmls_cnt_btrm,ccmls_cnt_ppty_stry,ccmls_cnt_unit,ccmls_dt_blt_yr,ccmls_nbr_gla,clmls_cnt_bdrm,clmls_cnt_tot_btrm,clmls_cnt_bldng_stry_cl,clmls_cnt_unit,...,ccmls_text_pub_lstg_cmnt,ccmls_desc_aplnc,ccmls_desc_eqpmnt_othr,sale_age_lag1,sale_age_toanchr,ccmls_dttm_mls_evnt,sale_age_toanchr_raw,final_statecounty_raw,final_state_raw,log_trgt
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.05, 0.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [53]:
##Create the input object to the NN
##Create input dictionaries for instantiating layers
## format {
ple_in = {col:dim for col,dim in zip([col.split('__')[1] for col in preprocessor.get_feature_names_out().tolist() if 'ple' in col],[ple_size]*len(x_cols_ple))}
std_in = {col:1 for col in [col.split('__')[1] for col in preprocessor.get_feature_names_out().tolist() if 'std_cols' in col]}
embdd_in = {col2 : len(gp_data[gp_data[col2].isna()==False][col2].unique())+1 for col2 in [col[0].split('embed_')[1] for col in preprocessor.transformers_ if 'embed' in col[0]] }
pssthrw_in = {col:1 for col in [col.split('__')[1] for col in preprocessor.get_feature_names_out().tolist() if 'slctr_passthrw' in col]}
input_dict = {**ple_in , **std_in , **embdd_in , **pssthrw_in ,**{'CLS':1}}

for key in ['final_address','ntdtefnd_lag1','final_ntdtefnd','clmls_dttm_list_orig_cl','clmls_desc_pub_rmrk','ccmls_text_pub_lstg_cmnt','ccmls_desc_aplnc','ccmls_desc_eqpmnt_othr','sale_age_lag1','sale_age_toanchr','ccmls_dttm_mls_evnt','sale_age_toanchr_raw','final_statecounty_raw','final_state_raw']:
    del input_dict[key]

print(input_dict)

{'ccmls_cnt_bdrm': 45, 'ccmls_cnt_btrm': 45, 'ccmls_cnt_ppty_stry': 45, 'ccmls_cnt_unit': 45, 'ccmls_dt_blt_yr': 45, 'ccmls_nbr_gla': 45, 'clmls_cnt_bdrm': 45, 'clmls_cnt_tot_btrm': 45, 'clmls_cnt_bldng_stry_cl': 45, 'clmls_cnt_unit': 45, 'clmls_cnt_grge_car': 45, 'clmls_dt_blt_yr': 45, 'clmls_nbr_area_livng': 45, 'clmls_cnt_prkng_spce_open': 45, 'clmls_cnt_prkng_spce_tot': 45, 'fre_cnt_unit': 45, 'fatax_amt_tot_asesd_val': 45, 'fatax_amt_asesd_imprmt_val': 45, 'fatax_amt_asesd_lnd_val': 45, 'fatax_cnt_bdrm': 45, 'fatax_cnt_bldng_stry': 45, 'fatax_cnt_prkng_spce': 45, 'fatax_cnt_tot_btrm_hve': 45, 'fatax_cnt_unit': 45, 'fatax_dt_eff_blt_yr': 45, 'fatax_nbr_livng_sq_ft': 45, 'fatax_nbr_lot_size_sq_ft': 45, 'fatax_amt_tot_mkt_val': 45, 'fatax_dt_cnsttn_yr': 45, 'fatax_dt_tax_yr': 45, 'idmtax_amt_asesd_tot_val': 45, 'idmtax_amt_asesd_imprmt_val': 45, 'idmtax_amt_asesd_lnd_val': 45, 'idmtax_cnt_bdrm': 45, 'idmtax_cnt_bldng_stry_drvd': 45, 'idmtax_cnt_grge_car': 45, 'idmtax_cnt_tot_btrm_hve

In [55]:
##save the input dictionary
##Save the prprocessor
with open(f'{artpath}/{state[0]}_{time}_ple{ple_size}_input_dict_new_h3_txt_extended.dill', 'wb') as dill_file:
    dill.dump(input_dict, dill_file)

In [56]:
##Clear memory from transform build
del preprocessor
del gp_data
del test_df
gc.collect()

963